In [2]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
df = pd.read_excel("./src/POM/T3/ex1.xlsx", sheet_name="exercises", engine="openpyxl")
df.columns = df.iloc[0] # load first row as column names
df = df.drop(df.index[0]) # drop first row
df

minimum_weekly = df.loc[df["Name"] == "Minimum Weekly Workout Time"]


,Name,Category,Sets,Set time in min,Break time between sets in min,Total time,Priority,NaN,Biceps,Chest,Triceps,Shoulder,Abdominals,Upper Back,Lower Back,Upper Thighs,Lower Thighs,Calves,Glutes,NaN
1,Cycling Warmup,warm up,1,7,0,7,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN
2,Rowing Warmup,warm up,1,7,0,7,4,NaN,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
3,Running Warmup,warm up,1,7,0,7,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN
4,Abduction Machine,resistance,3,1,1.5,6,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,0,NaN
5,Adduction Machine,resistance,3,1,1.5,6,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,2,NaN
6,Barbell Row,resistance,3,1,1.5,6,3,NaN,1,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
7,Bench Press,resistance,4,1,2,10,5,NaN,NaN,2,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Benchpress Declined Dunbells,resistance,3,1,1.5,6,3,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Benchpress Inclined Dunbells,resistance,3,1,1.5,6,4,NaN,NaN,1,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Biceps Curls Barbell,resistance,3,1,1.5,6,2,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df = pd.read_excel("ex1.xlsx", sheet_name="info")
df


,Parameter,Value
0,"Number of days in a ""week""",7
1,Number of weeks of workout schedules,3
2,Time for preparation before and after the workout,25
3,Minimum workout time (incl. breaks) in minutes,45
4,Maximum workout time (incl. breaks) in minutes,100
5,Break time between exercises in minutes,5
6,Minimum number of workouts per week,2
7,Maximum number of workouts per week,4
8,Earliest hour to start,8
9,Latest hour to be finished,22


In [84]:
import test

weekLen, weekNum, prep, minWork, maxWork, pause, minWeek, maxWeek, early, latest, startDay, startMonth, startYear, zone, allExercise, needs = test.prepare_input("ex1.xlsx")
print("weekLen: ", weekLen)
print("weekNum: ", weekNum)
print("prep: ", prep)
print("minWork: ", minWork)
print("maxWork: ", maxWork)
print("pause: ", pause)
print("minWeek: ", minWeek)
print("maxWeek: ", maxWeek)
print("early: ", early)
print("latest: ", latest)
print("startDay: ", startDay)
print("startMonth: ", startMonth)
print("startYear: ", startYear)
print("zone: ", zone)
print(f"allExercise {len(allExercise)}: {allExercise}")
print("needs: ", needs)


weekLen:  7
weekNum:  3
prep:  25
minWork:  45
maxWork:  100
pause:  5
minWeek:  2
maxWeek:  4
early:  8
latest:  22
startDay:  20
startMonth:  7
startYear:  2023
zone:  2
allExercise 46: [{'name': 'Cycling Warmup', 'category': 'warm up', 'setTime': 7, 'priority': 4, 'biceps': -1, 'chest': -1, 'triceps': -1, 'shoulder': -1, 'abdominal': -1, 'backUp': -1, 'backLow': -1, 'thighUp': 0, 'thighLow': 0, 'calves': 0, 'glutes': -1}, {'name': 'Rowing Warmup', 'category': 'warm up', 'setTime': 7, 'priority': 4, 'biceps': 0, 'chest': -1, 'triceps': -1, 'shoulder': -1, 'abdominal': -1, 'backUp': 0, 'backLow': 0, 'thighUp': -1, 'thighLow': -1, 'calves': -1, 'glutes': -1}, {'name': 'Running Warmup', 'category': 'warm up', 'setTime': 7, 'priority': 4, 'biceps': -1, 'chest': -1, 'triceps': -1, 'shoulder': -1, 'abdominal': -1, 'backUp': -1, 'backLow': -1, 'thighUp': 0, 'thighLow': 0, 'calves': 0, 'glutes': 0}, {'name': 'Abduction Machine', 'category': 'resistance', 'setTime': 6, 'priority': 3, 'biceps'

### Write Up on the constraints

The objective is the maximization of the total sum of priorities over all
performed exercises within the optimization time frame

**Workouts**
- Workout routine comprises of one or multiple weeks, choosen by customer
- At max **1 Workout** per day and always in the largest possible free time slot
- 3 Types of exercises: warm up, cardio, resistance, each workout has exactly one warm up
- Between exercises: constant break
- Before and after workout: constant prep time each
- exercises include different body parts: there must be breaks (consecutive days) between exercising body parts
- Resting days for a body part: use maximum value of exercise

**Data constraint**
- given first day and weeks to optimize
- time frame for each day
- maximum and minimum workouts per week
- miminum workout for each body part (in minutes)
- some exercises don't require resting a body part, however, cannot be executed
if already resting on that body part


**Danger**
- Calendar has past events, must be considered for resting days
- user data is timezone dependent

In [4]:
import dataclasses
import pytz
import datetime
import pandas as pd
import re

body_parts_names = ["Biceps","Chest","Shoulder","Triceps","Abdominals", "Upper Back","Lower Back", "Upper Thighs","Lower Thighs", "Calves","Glutes"]


@dataclasses.dataclass
class ExPlan: 
    name: str                # exercise name 
    category: str            # exercise category
    _sets: int                # number of sets
    _perSetTime: int          # time per set in minutes
    _breakTime: int           # break time between sets in minutes
    totalTime: int           # total time of exercise in minutes
    priority: int            # priority of exercise, higher is more important
    body_rest: dict[str, int] # days of rest after exercise for each muscle group

    def parse_body_part(self, frame: pd.DataFrame, attr: str):
        if isinstance(frame[attr], int):
            self.body_rest[attr] = frame[attr]
        else:
            self.body_rest[attr] = -1 # if not any meaningfull data type, we just say it is -1 

    def __init__(self, df: pd.DataFrame):
        self.body_rest = {}
        self.parse_body_part(df, "Biceps")
        self.parse_body_part(df, "Chest")
        self.parse_body_part(df, "Shoulder")
        self.parse_body_part(df, "Triceps")
        self.parse_body_part(df, "Abdominals")
        self.parse_body_part(df, "Upper Back")
        self.parse_body_part(df, "Lower Back")
        self.parse_body_part(df, "Upper Thighs")
        self.parse_body_part(df, "Lower Thighs")
        self.parse_body_part(df, "Calves")
        self.parse_body_part(df, "Glutes")
        self.name = df["Name"]
        self.category = df["Category"].lower().replace(" ", "")
        self.totalTime = df["Total time"] # Todo: sanity check against other time values
        self.priority = df["Priority"]
        self._sets = -1 # Todo: sanity check against other time values
        self._perSetTime = -1 
        self._breakTime = -1


    def __repr__(self) -> str:
        # pretty print the class with line breaks and attribute names
        return f"ExPlan(\n\tname={self.name},\n\t_sets={self._sets},\n\t_perSetTime={self._perSetTime},\n\t_breakTime={self._breakTime},\n\ttotalTime={self.totalTime},\n\tpriority={self.priority},\n\tbody_rest={self.body_rest})"
    
    # overload adding for calculation of max rest
    def __add__(self, other):
        dominant_exercise = ExPlan(f"{self.name}&{other.name}")
        for body_part in body_parts_names:
            dominant_exercise.body_rest[body_part] = max(self.body_rest[body_part], other.body_rest[body_part])


@dataclasses.dataclass
class NeedsPlan:
    body_need = dict[str, int] # minimum body training time needed for each muscle group

    def parse_value(self, frame: pd.DataFrame, attr: str):
        if isinstance(frame[attr], int):
            self.body_need[attr] = frame[attr]
        else:
            self.body_need[attr] = 0 # if not any meaningfull data type, we just say it is 0
        
    # a single row!
    def __init__(self, exercise_dataset: pd.DataFrame):
        self.body_need = {}
        self.parse_value(exercise_dataset, "Biceps")
        self.parse_value(exercise_dataset, "Chest")
        self.parse_value(exercise_dataset, "Shoulder")
        self.parse_value(exercise_dataset, "Triceps")
        self.parse_value(exercise_dataset, "Abdominals")
        self.parse_value(exercise_dataset, "Upper Back")
        self.parse_value(exercise_dataset, "Lower Back")
        self.parse_value(exercise_dataset, "Upper Thighs")
        self.parse_value(exercise_dataset, "Lower Thighs")
        self.parse_value(exercise_dataset, "Calves")
        self.parse_value(exercise_dataset, "Glutes")


    def __repr__(self) -> str:
        # pretty print the class with line breaks and attribute names
        return f"NeedsPlan(\n\tbiceps={self.body_need})"

@dataclasses.dataclass
class ExInfo: # describes the full exercise plan
    weekLen: int            # length of a week in days
    weekNum: int            # number of weeks to plan 
    prep: int               # preparation time in minutes, before and after workout
    minWork: int            # minimum time a workout, not including prep (before and after), including pause
    maxWork: int            # maximum time a workout, not including prep (before and after), including pause
    pause: int              # pause between workouts in minutes
    minWeek: int            # minimum number of workouts per week
    maxWeek: int            # maximum number of workouts per week
    early: int              # earliest hour of day to start a workout (inclusive prep)
    latest: int             # latest hour of day to start a workout (inclusive prep)
    startDay: datetime.date # day to start the plan
    zone: datetime.timezone # timezone of the plan
    allExercise: dict[str, ExPlan]   # exercises by names
    allExerciseByBody: dict[str, list[str]] # exercises by body part
    needs: NeedsPlan        # needs to play the plan


    def __init__(self, ex_path):
        info_dataset = pd.read_excel(ex_path, sheet_name="info")
        self.sanity_check_info(info_dataset)
        exercise_dataset = pd.read_excel(ex_path, sheet_name="exercises")

        exercise_dataset.columns = exercise_dataset.iloc[0] # load first row as column names
        exercise_dataset = exercise_dataset.drop(exercise_dataset.index[0]) # drop first row

        self.allExercise = {}
        self.allExerciseByBody = {}
        self.needs = None
        self.sanity_check_needs(exercise_dataset)

        # drop last row
        exercise_dataset = exercise_dataset.drop(exercise_dataset.index[-1])
        self.sanity_check_exercise(exercise_dataset)
        self.sanity_check_exercise_by_body(exercise_dataset)


    def sanity_check_needs(self, exercise_dataset: pd.DataFrame):
        last_row = exercise_dataset.iloc[-1]
        assert last_row["Name"] == "Minimum Weekly Workout Time", "last row of exercise dataset is not minimum weekly workout time"
        self.needs = NeedsPlan(last_row)

    def sanity_check_exercise(self, exercise_dataset: pd.DataFrame):
        for index, row in exercise_dataset.iterrows():
            category: str = row["Category"]
            if pd.isnull(category) or pd.isnull(row["Name"]): # exercise without names or category sucks
                assert False, f"exercise {row} has no name or category"
            self.allExercise[row["Name"]] = ExPlan(row)
    
    def sanity_check_exercise_by_body(self, exercise_dataset: pd.DataFrame):
        for index, row in exercise_dataset.iterrows():
            category: str = row["Category"]
            if pd.isnull(category) or pd.isnull(row["Name"]):
                assert False, f"exercise {row} has no name or category"
            for i, v in row.items():
                if i in body_parts_names and (isinstance(v, int) or isinstance(v, float))  and v >= 0 :
                    if i in self.allExerciseByBody:
                        self.allExerciseByBody[i].append(row["Name"])
                    else:
                        self.allExerciseByBody[i] = [row["Name"]]
        
    def sanity_check_info(self, info_dataset: pd.DataFrame):
        assert info_dataset.shape == (12, 2), "info dataset has unexpected shape"
        # A table of Parameter: Value, we are guaranteed that the parameters stay the same

        assert isinstance(info_dataset.iloc[0, 1], int), "weekLen is not an int"
        assert info_dataset.iloc[0, 1] >= 0, f"unexpected weekLen of {info_dataset.iloc[0, 1]}"
        assert info_dataset.iloc[0, 1] <= 7, f"unexpected weekLen of {info_dataset.iloc[0, 1]}"
        self.weekLen = info_dataset.iloc[0, 1]
        assert isinstance(info_dataset.iloc[1, 1], int), "weekNum is not an int"
        assert info_dataset.iloc[1, 1] >= 0, f"unexpected weekNum of {info_dataset.iloc[1, 1]}"
        self.weekNum = info_dataset.iloc[1, 1]
        assert isinstance(info_dataset.iloc[2, 1], int), "prep is not an int"
        assert info_dataset.iloc[2, 1] >= 0, f"unexpected prep of {info_dataset.iloc[2, 1]}"
        self.prep = info_dataset.iloc[2, 1]
        assert isinstance(info_dataset.iloc[3, 1], int), "minWork is not an int"
        assert info_dataset.iloc[3, 1] >= 0, f"unexpected minWork of {info_dataset.iloc[3, 1]}"
        self.minWork = info_dataset.iloc[3, 1]
        assert isinstance(info_dataset.iloc[4, 1], int), "maxWork is not an int"
        assert info_dataset.iloc[4, 1] >= 0, f"unexpected maxWork of {info_dataset.iloc[4, 1]}"
        assert info_dataset.iloc[4, 1] >= self.minWork, f"maxWork {info_dataset.iloc[4, 1]} smaller than minWork {self.minWork}"
        self.maxWork = info_dataset.iloc[4, 1]
        assert isinstance(info_dataset.iloc[5, 1], int), "pause is not an int"
        assert info_dataset.iloc[5, 1] >= 0, f"unexpected pause of {info_dataset.iloc[5, 1]}"
        self.pause = info_dataset.iloc[5, 1]
        assert isinstance(info_dataset.iloc[6, 1], int), "minWeek is not an int"
        assert info_dataset.iloc[6, 1] >= 0, f"unexpected minWeek of {info_dataset.iloc[6, 1]}"
        self.minWeek = info_dataset.iloc[6, 1]
        assert isinstance(info_dataset.iloc[7, 1], int), "maxWeek is not an int"
        assert info_dataset.iloc[7, 1] >= 0, f"unexpected maxWeek of {info_dataset.iloc[7, 1]}"
        assert info_dataset.iloc[7, 1] >= self.minWeek, f"maxWeek {info_dataset.iloc[7, 1]} smaller than minWeek {self.minWeek}"
        self.maxWeek = info_dataset.iloc[7, 1]
        assert isinstance(info_dataset.iloc[8, 1], int), "early is not an int"
        assert info_dataset.iloc[8, 1] >= 0, f"unexpected early of {info_dataset.iloc[8, 1]}"
        assert info_dataset.iloc[8, 1] <= 24, f"unexpected early of {info_dataset.iloc[8, 1]}"
        self.early = info_dataset.iloc[8, 1]
        assert isinstance(info_dataset.iloc[9, 1], int), "latest is not an int"
        assert info_dataset.iloc[9, 1] >= 0, f"unexpected latest of {info_dataset.iloc[9, 1]}"
        assert info_dataset.iloc[9, 1] <= 24, f"unexpected latest of {info_dataset.iloc[9, 1]}"
        assert info_dataset.iloc[9, 1] >= self.early, f"latest {info_dataset.iloc[9, 1]} smaller than early {self.early}"
        self.latest = info_dataset.iloc[9, 1]
        assert isinstance(info_dataset.iloc[10, 1], datetime.date), "startDay is not a date"
        assert isinstance(info_dataset.iloc[11, 1], str), "zone is not a string"
        # remove spaces from zone
        zone = info_dataset.iloc[11, 1].replace(" ", "")
        # assuming UTC+X format
        offset_hours = int(re.search(r'\d+', zone).group())
        tz = datetime.timezone(datetime.timedelta(hours=offset_hours))
        self.zone = tz
        self.startDay = info_dataset.iloc[10, 1].replace(tzinfo=datetime.timezone.utc).astimezone(tz=self.zone)
        self.startDay = self.startDay.replace(hour=0)

    
    def __repr__(self) -> str:
        # pretty print the class with line breaks and attribute names
        return f"ExInfo(\n\tweekLen={self.weekLen},\n\tweekNum={self.weekNum},\n\tprep={self.prep},\n\tminWork={self.minWork},\n\tmaxWork={self.maxWork},\n\tpause={self.pause},\n\tminWeek={self.minWeek},\n\tmaxWeek={self.maxWeek},\n\tearly={self.early},\n\tlatest={self.latest},\n\tstartDay={self.startDay},\n\tzone={self.zone},\n\tallExercise={self.allExercise},\n\tneeds={self.needs}\n)"

ex_1 = ExInfo("ex1.xlsx")


In [5]:
import icalendar
import dataclasses
import datetime
import pytz



@dataclasses.dataclass
class UserEvent:
    event: str
    isExercise: bool
    exercises: list[str]
    start: datetime.datetime
    end: datetime.datetime
    stamp: datetime.datetime
    def __init__(self, event, start, end, stamp, isExercise=False, exercises=[]):
        self.event = event
        self.start = start
        self.end = end
        self.stamp = stamp
        self.isExercise = isExercise
        self.exercises = exercises
    
    def __repr__(self) -> str:
        # pretty print the class with line breaks and attribute names
        return f"UserEvent(\n\tevent={self.event},\n\tisExercise={self.isExercise},\n\texercises={self.exercises},\n\tstart={self.start},\n\tend={self.end},\n\tstamp={self.stamp}\n)"

@dataclasses.dataclass
class DayEventIterator: 
    events: list[UserEvent]
    def __init__(self, events: list[UserEvent]):
        self.index = 0
        self.events = events
    
    def next(self):
        # returns list of events grouped by day
        if self.index < len(self.events):
            day = self.events[self.index].start.date()
            day_events = []
            while self.index < len(self.events) and self.events[self.index].start.date() == day:
                day_events.append(self.events[self.index])
                self.index += 1
            return day_events
        return None

@dataclasses.dataclass
class UserCalendar: 
    events: list[UserEvent] 
    tz_shift: datetime.timezone
    day_iter: DayEventIterator
    def __init__(self, tz, start_date = datetime.date.today()):
        self.tz_shift = tz
        self.events = []
        self.start_date = start_date

    # note: Fixed timezones!
    def load_calendar(self, cpath):
        file = open(cpath)
        cal = icalendar.Calendar.from_ical(file.read())
        for component in cal.walk():
            if component.name == "VEVENT":
                ds: datetime.datetime = component.decoded("dtstart")
                dt: datetime.datetime = component.decoded("dtend")
                dst: datetime.datetime = component.decoded("dtstamp")
                ds = ds.replace(tzinfo=datetime.timezone.utc).astimezone(tz=self.tz_shift)
                dt = dt.replace(tzinfo=datetime.timezone.utc).astimezone(tz=self.tz_shift)
                dst= dst.replace(tzinfo=datetime.timezone.utc).astimezone(tz=self.tz_shift)
                event: str = str(component.decoded("summary").decode("utf-8") )
                event = event.lower().replace(" ","")
                if event.startswith("workout"):
                    desc = component.get("description")
                    desc = [x.strip() for x in desc.split(',')[0].split("\n")]
                    self.events.append(UserEvent(
                        event = event,
                        start = ds,
                        end = dt, 
                        stamp = dst,
                        isExercise = True,
                        exercises = desc
                    ))
                else:
                    self.events.append(UserEvent(
                        event = event,
                        start = ds,
                        end = dt, 
                        stamp = dst
                    ))
        self.events.sort(key=lambda x: x.start) 
        self.day_iter = DayEventIterator(self.events)
        file.close()




    
    def __repr__(self) -> str:
        # pretty print the class with line breaks and attribute names
        return f"UserCalendar(\n\tevents={self.events},\n\ttz_shift={self.tz_shift}\n)"

c = UserCalendar(datetime.timezone(datetime.timedelta(hours=2)), ex_1.startDay)
c.load_calendar("cal1.ics")
suma = 0
for day_events in c.day_iter:
    suma += len(day_events)
    print(f"There were {len(day_events)} events on day {day_events[0].start.date()}")

assert suma == len(c.events), "not all events were iterated over"
print("Total Events: ", len(c.events))

There were 7 events on day 2023-07-13
There were 7 events on day 2023-07-14
There were 1 events on day 2023-07-15
There were 1 events on day 2023-07-16
There were 4 events on day 2023-07-17
There were 5 events on day 2023-07-18
There were 4 events on day 2023-07-19
There were 3 events on day 2023-07-20
There were 5 events on day 2023-07-21
There were 4 events on day 2023-07-24
There were 6 events on day 2023-07-25
There were 5 events on day 2023-07-26
There were 7 events on day 2023-07-27
There were 6 events on day 2023-07-28
There were 1 events on day 2023-07-29
There were 6 events on day 2023-07-31
There were 7 events on day 2023-08-01
There were 6 events on day 2023-08-02
There were 5 events on day 2023-08-03
There were 6 events on day 2023-08-04
There were 5 events on day 2023-08-07
There were 4 events on day 2023-08-08
There were 5 events on day 2023-08-09
Total Events:  110


In [85]:
def calc_collision_pairs(info: ExInfo) -> dict[str, list[(str, int)]]:
    blocking_exercises = {}
    for ex in info.allExercise.values():
        for body_part, rest_days in ex.body_rest.items():
            if rest_days > 0:
                # print(f"Performing {ex.name} must Rest {body_part} for {rest_days} days blocking exercises: {info.allExerciseByBody[body_part]}")
                for blocking_ex in info.allExerciseByBody[body_part]:
                    if ex.name in blocking_exercises:
                        if blocking_ex in blocking_exercises[ex.name]:
                            blocking_exercises[ex.name][blocking_ex] = max(rest_days,blocking_exercises[ex.name][blocking_ex])
                        else:
                            blocking_exercises[ex.name][blocking_ex] = rest_days
                    else:
                        blocking_exercises[ex.name] = {blocking_ex: rest_days}
    return blocking_exercises

def calc_day_timeslot(exercise_plan: ExInfo, day_events: list[UserEvent]) -> int:
    #print(f"{day_events[0].stamp.date()} events {[x.event for x in day_events]}")
    # we precheck against the earliest first
    earliest_start_time = datetime.datetime.combine(
        day_events[0].start.date(), 
        datetime.time(hour=exercise_plan.early, minute=0, second=0))
    earliest_start_time = earliest_start_time.replace(tzinfo=exercise_plan.zone)
    max_timeslot = (day_events[0].start - earliest_start_time).total_seconds() / 60
    gap = ["earliest", day_events[0].event]
    for previous_event, current_event in zip(day_events, day_events[1:]):
        diff_min = (current_event.start - previous_event.end).total_seconds() / 60
        if diff_min > max_timeslot:
            gap = [previous_event.event, current_event.event]
            max_timeslot = diff_min
    # we also check against the latest
    latest_end_time = datetime.datetime.combine(
        day_events[-1].start.date(),
        datetime.time(hour=exercise_plan.latest, minute=59, second=00))
    latest_end_time = latest_end_time.replace(tzinfo=exercise_plan.zone)
    diff_min = (latest_end_time - day_events[-1].end).total_seconds() / 60
    if diff_min > max_timeslot:
        max_timeslot = diff_min
        gap = [day_events[-1].event, "latest"]
    
    #print(f"max_timeslot of day {day_events[0].stamp.date()}: {max_timeslot}min in beween {gap[0]} and {gap[1]}")
    return max_timeslot

def build_simple_model(info: ExInfo, cal: UserCalendar):
    import json
    import gurobipy as gp
    blocking_exercises = calc_collision_pairs(info)
    model = gp.Model("workout")
    days = [i for i in range(1, info.weekLen * info.weekNum + 1)]

    # should train on day
    d = {}
    for day_c in days:
        d[day_c] = model.addVar(vtype=gp.GRB.BINARY, obj=0.0, name=f"d_{day_c}")

    # variables for exercise selection
    x = {} 
    for day_c in days:
        for ex in info.allExercise:
            x[f"{ex}_{day_c}"] = model.addVar(vtype=gp.GRB.BINARY, obj=1.0, name=f"x_{ex}_{day_c}")

    # training time of body part per day
    training_time = {}
    for day_c in days:
        for body_part in body_parts_names:
            training_time[f"{body_part}_{day_c}"] = model.addVar(vtype=gp.GRB.CONTINUOUS, obj=0.0, name=f"t_{body_part}_{day_c}")

    # constr: only train if we have exercises selected
    for day_c in days:
        for ex in info.allExercise:
            model.addConstr(x[f"{ex}_{day_c}"] <= d[day_c], name=f"only_train_if_day_{ex}_{day_c}")

    # constr: we do excactly one warmup if we train on that day
    warumups = [w for w in info.allExercise if info.allExercise[w].category == "warmup"]
    print(warumups)
    for day_c in days:
        model.addConstr(gp.quicksum(x[f"{ex}_{day_c}"] for ex in warumups) == d[day_c], name=f"only_one_warmup_{day_c}")
    
    # constr: if we perform an exercise, we add the exercise time to the training time of the body part
    for day_c in days:
        for body_part, exercises in info.allExerciseByBody.items():
            model.addConstr(
                gp.quicksum(x[f"{ex}_{day_c}"] * info.allExercise[ex].totalTime for ex in exercises)
                == 
                training_time[f"{body_part}_{day_c}"],
                name=f"training_time_{body_part}_{day_c}"
            )

    # constr: consider the minimum and maximum training time for each workout 
    for day_c in days:
        model.addConstr(
            gp.quicksum(
                x[f"{ex}_{day_c}"] * (info.allExercise[ex].totalTime + info.pause) for ex in info.allExercise
            ) - info.pause * d[day_c] 
            >= info.minWork * d[day_c], 
            name=f"min_work_{day_c}")
        
        model.addConstr(
            gp.quicksum(
                x[f"{ex}_{day_c}"] * (info.allExercise[ex].totalTime + info.pause) for ex in info.allExercise
            ) - info.pause * d[day_c]
            <= info.maxWork * d[day_c], 
            name=f"max_work_{day_c}")

    # constr: for each week, we need to train at least minWeek and at most maxWeek 
    # constr: also we have to consider per body part training times
    for week in range(0, info.weekNum): 
        days_of_week = [i for i in range(week * info.weekLen + 1, week * info.weekLen + info.weekLen + 1)]
        print(f"days_of_week: {days_of_week}")
        model.addConstr(gp.quicksum(d[day] for day in days_of_week) >= info.minWeek, name=f"min_week_{week}")
        model.addConstr(gp.quicksum(d[day] for day in days_of_week) <= info.maxWeek, name=f"max_week_{week}")
        for body_part, time in info.needs.body_need.items():
            model.addConstr(gp.quicksum(training_time[f"{body_part}_{day}"] for day in days_of_week) >= time, name=f"min_week_{week}_{body_part}")


    # general training rule

    # constr: we need to consider the calendar and find free time slots
    print(f"startDay: {info.startDay}") 
    block_on_day = {} # starting with 0 for startday
    visited_days = set()
    for day_events in cal.day_iter:
        this_day = day_events[0].start.date()
        diff_days = (this_day - info.startDay.date()).days
        if diff_days < 0: # if event is before startDay, we only collect exercises
            for event in day_events:
                if event.isExercise: # event was an exercise
                    print(f"Prior Exercise on {event.start.date()}: {event.exercises}")
                    for exercise in event.exercises: 
                        if exercise in blocking_exercises:
                            for blocking_ex, rest_days in blocking_exercises[exercise].items():
                                if  rest_days + diff_days >= 0: # block exercise if it will our plan
                                    if rest_days + diff_days in block_on_day:
                                        block_on_day[rest_days + diff_days].add(blocking_ex)
                                    else:
                                        block_on_day[rest_days + diff_days] = {blocking_ex}
        else:  # we are at start of exercise plan phase
            max_workout_time = calc_day_timeslot(info, day_events)
            print(f"Max Workout Time on {this_day}: {max_workout_time} with {2*info.prep} prep time and {info.pause} pause time")
            visited_days.add(diff_days+1)
            model.addConstr(
                gp.quicksum(
                    x[f"{ex}_{diff_days+1}"] * (info.allExercise[ex].totalTime + info.pause) for ex in info.allExercise
                ) - info.pause * d[diff_days + 1] + info.prep * d[diff_days+1]
                <= max_workout_time * d[diff_days + 1],
                name=f"max_workout_time_{diff_days+1}"

            )
     

    # constr: we need to consider the block on day block
    print(block_on_day)
    for day_c, blocking_exs in block_on_day.items():
        for blocking_ex in blocking_exs:
            model.addConstr(
                x[f"{blocking_ex}_{day_c+1}"] == 0,
                name=f"block_on_day_{blocking_ex}_{day_c+1}"
            )
    max_training_time = (info.latest - info.early) * 60
    max_training_time += 59 
    for day_c in days:
        if day_c not in visited_days:
            model.addConstr(
                gp.quicksum(
                    x[f"{ex}_{day_c}"] * (info.allExercise[ex].totalTime + info.pause) for ex in info.allExercise
                ) - info.pause * d[day_c] + info.prep * d[day_c]
                <= max_training_time * d[day_c],
                name=f"max_workout_time_{day_c}"
            )
    # constr: now add days that were not visited
    for day_c in days:
        if day_c >= info.weekLen * info.weekNum: 
            continue # dont care about last day
        for ex, blocking_exs in blocking_exercises.items():
            for blocking_ex, rest_days in blocking_exs.items():
                if rest_days <= 0: # if rest is 0, we dont need to block
                    continue
                ran = []
                if day_c + 1 + rest_days > info.weekLen * info.weekNum: # if rest would go out of timespan, we just block until end
                    ran = [t for t in range(day_c + 1, info.weekLen * info.weekNum + 1)] 
                else: # else we block until rest days are over starting from next day
                    ran = [t for t in range(day_c + 1, day_c + rest_days + 1 )]
                # print(f"Blocking {blocking_ex} for {ran} after {ex}")
                model.addConstr(
                    gp.quicksum(x[f"{blocking_ex}_{d}"] for d in ran)
                    <= (1 - x[f"{ex}_{day_c}"]),
                    name=f"block_{blocking_ex}_{ex}_{day_c}"
                )

    model.update()
    # objective: maximize priority
    model.setObjective(gp.quicksum(x[f"{ex}_{day}"] * info.allExercise[ex].priority for ex in info.allExercise for day in days), gp.GRB.MAXIMIZE)
    model.write("workout.lp")
    model.update()
    model.optimize() # todo: find mistake obj is off
    debug = True
    if model.status == gp.GRB.OPTIMAL:
        print("Instance is feasible. Best objective value: ", model.objVal)
        if debug:
            for trainings, v in x.items():
                if v.x > 0.8:
                    training = trainings.split("_")
                    body_parts_used = [(b, r) for b,r in info.allExercise[training[0]].body_rest.items() if r > 0]
                    print(f"Day {training[1]}: {training[0]} is selected ({body_parts_used})")
    else:
        print("Instance is infeasible.")

ex = ExInfo("ex2.xlsx")
cal = UserCalendar(ex.zone) 
cal.load_calendar("cal2.ics")
build_simple_model(ex, cal)

['Cycling Warmup', 'Running Warmup', 'Stepping Warmup']
days_of_week: [1, 2, 3, 4, 5, 6, 7]
days_of_week: [8, 9, 10, 11, 12, 13, 14]
startDay: 2023-12-03 00:00:00+01:00
Prior Exercise on 2023-11-26: ['Rowing Warmup', 'Lat Pull', 'Face Pulls', 'Cable Row Narrow', 'Cable Row Wide', 'Biceps Curls Barbell']
Prior Exercise on 2023-11-28: ['Running Warmup', 'Bench Press', 'Butterfly', 'Shoulder Press', 'Triceps Rope Pulldown']
Prior Exercise on 2023-12-01: ['Running Warmup', 'Squat', 'Lunges', 'Leg Extension Machine', 'Leg Curls Sitting', 'Standing Calf Raise Machine']
Max Workout Time on 2023-12-04: 329.0 with 50 prep time and 5 pause time
Max Workout Time on 2023-12-05: 329.0 with 50 prep time and 5 pause time
Max Workout Time on 2023-12-06: 329.0 with 50 prep time and 5 pause time
Max Workout Time on 2023-12-07: 409.0 with 50 prep time and 5 pause time
Max Workout Time on 2023-12-08: 329.0 with 50 prep time and 5 pause time
Max Workout Time on 2023-12-11: 120.0 with 50 prep time and 5 pau

In [80]:
ex = ExInfo("ex1.xlsx")
colls = calc_collision_pairs(ex)
w = ['Rowing Warmup', 'Lat Pull', 'Face Pulls', 'Cable Row Narrow', 'Cable Row Wide', 'Biceps Curls Barbell'] 
block_day_one= set()
for e in w:
    if e in colls: 
        for ele,c in colls[e].items():
            if c >= 2:
                block_day_one.add(ele) 
print(block_day_one)
print(colls["Butterfly"])

sol = ['Rowing Warmup', 'Biceps Curls Dunbells', 'Cable Row Narrow', 'Cable Row Wide', 'Core Workout', 'Deadlift', 'Incline Biceps Curl', 'Lat Pull', 'Lunges'] 
extime = 0
for s in sol:
    extime += ex.allExercise[s].totalTime
extime += ex.pause * (len(sol) - 1)
print(f"{extime} < {ex.maxWork}")

print(ex.allExercise["Triceps Overhead Pull"].totalTime)

{'Swimming', 'Cable Row Wide', 'Prisoner Back Extensions', 'HIIT', 'Rowing Warmup', 'Butterfly Reverse', 'Lat Pull', 'Cable Row Narrow', 'Incline Biceps Curl', 'Deadlift', 'Pullups', 'Barbell Row', 'Biceps Curls Dunbells', 'Face Pulls', 'Biceps Curls Barbell', 'Hammer Curls'}
{'Bench Press': 1, 'Benchpress Declined Dunbells': 1, 'Benchpress Inclined Dunbells': 1, 'Butterfly': 1, 'Cable Crossover': 1, 'Dips': 1, 'Overhead Press': 1, 'Pullups': 1, 'Pushups': 1, 'Shoulder Press': 1, 'Shoulder Press Machine': 1, 'HIIT': 1}
98.5 < 100
6


In [7]:

def calc_collision_pairs(info: ExInfo) -> dict[str, list[(str, int)]]:
    blocking_exercises = {}
    for ex in info.allExercise.values():
        for body_part, rest_days in ex.body_rest.items():
            if rest_days >= 0:
                # print(f"Performing {ex.name} must Rest {body_part} for {rest_days} days blocking exercises: {info.allExerciseByBody[body_part]}")
                for blocking_ex in info.allExerciseByBody[body_part]:
                    if ex.name in blocking_exercises:
                        if blocking_ex in blocking_exercises[ex.name]:
                            blocking_exercises[ex.name][blocking_ex] = max(rest_days,blocking_exercises[ex.name][blocking_ex])
                        else:
                            blocking_exercises[ex.name][blocking_ex] = rest_days
                    else:
                        blocking_exercises[ex.name] = {blocking_ex: rest_days}
    return blocking_exercises

ex = ExInfo("ex2.xlsx")
cal = UserCalendar(ex.zone) 
cal.load_calendar("cal2.ics")
blocking_exercises = calc_collision_pairs(ex)
blocking_exercises["Abduction Machine"]

{'Cycling Warmup': 1,
 'Running Warmup': 1,
 'Stepping Warmup': 1,
 'Abduction Machine': 1,
 'Adduction Machine': 1,
 'Deadlift': 1,
 'Leg Curls Laying': 1,
 'Leg Curls Sitting': 1,
 'Leg Extension Machine': 1,
 'Lunges': 1,
 'Squat': 1,
 'Cycling': 1,
 'HIIT': 1,
 'Running': 1,
 'Zumba': 1,
 'Hip Raises': 0}

In [8]:
# calculates the maximum continuous timeslot in minutes available in a day for a workout
# we assume that the events are sorted by start time, because we presorted them
# we also assume that the events are in the same day
def calc_day_timeslot(exercise_plan: ExInfo, calendar: UserCalendar, day_index: int) -> int:
    #print(f"{day_events[0].stamp.date()} events {[x.event for x in day_events]}")
    # we precheck against the earliest first
    earliest_start_time = datetime.datetime.combine(
        day_events[0].start.date(), 
        datetime.time(hour=exercise_plan.early))
    earliest_start_time = earliest_start_time.replace(tzinfo=exercise_plan.zone)
    max_timeslot = (day_events[0].start - earliest_start_time).total_seconds() / 60
    gap = ["earliest", day_events[0].event]
    for previous_event, current_event in zip(day_events, day_events[1:]):
        diff_min = (current_event.start - previous_event.end).total_seconds() / 60
        if diff_min > max_timeslot:
            gap = [previous_event.event, current_event.event]
            max_timeslot = diff_min
    # we also check against the latest
    latest_end_time = datetime.datetime.combine(
        day_events[-1].start.date(),
        datetime.time(hour=exercise_plan.latest))
    latest_end_time = latest_end_time.replace(tzinfo=exercise_plan.zone)
    diff_min = (latest_end_time - day_events[-1].end).total_seconds() / 60
    if diff_min > max_timeslot:
        max_timeslot = diff_min
        gap = [day_events[-1].event, "latest"]
    
    #print(f"max_timeslot of day {day_events[0].stamp.date()}: {max_timeslot}min in beween {gap[0]} and {gap[1]}")
    return max_timeslot


def build_graph(calendar: UserCalendar, exercise: ExInfo):
    print("startDay ", exercise.startDay) 
    print(f"train for {exercise.weekNum} total days with {exercise.weekLen} days per week")
    blocked_exercises: dict[int, list[str]] = {} # day (from startDay) -> list of exercises blocked
    graph = {}
    # iterate over days in calendar
    for day_events in calendar.day_iter:
        this_day = day_events[0].start.date()
        if this_day < exercise.startDay.date(): # if event is before startDay, we only collect exercises
            for event in day_events:
                if event.isExercise:
                    print(f"Prior Exercise on {event.start.date()}: {event.exercises}")
        else:  # we are at start of exercise plan phase
            max_workout_time = calc_day_timeslot(day_events, exercise)
            if max_workout_time <= exercise.minWork + 2 * exercise.prep: # must consider prep time before and after
                print(f"Day {this_day} has not enough time for a workout ({max_workout_time}min < {exercise.minWork + 2 * exercise.prep}min)")
                continue 
            
            # print(f"Day {this_day} has enough time for a workout ({max_workout_time}min > {exercise.minWork + 2 * exercise.prep }min)")
            day_exercises = []


In [9]:
import json
# Graph: Exercise -> Exercise: (Cost/priority, Time)
# Exercise Plan Graph of one day
def build_day_graph(exercise: ExInfo) -> tuple[dict[str, dict[str, (int, int)]], dict[str, dict[str, (int, int)]]]:
    graph = {}
    graph_r = {} # reverse graph, incoming edges
    # we always start with some warmup
    warumups = [warmup for warmup in exercise.allExercise.values() if warmup.category == "warmup"]
    graph["day_start"] = {warmup.name: (0,0) for warmup in warumups}
    graph_r["day_start"] = {}
    for warmup in warumups:
        graph[warmup.name] = {}
        graph_r[warmup.name] = {}
        graph_r[warmup.name]["day_start"] = (0,0)
    # next, we create a allExercise * (maxWork / (pause )) sized graph with cross connected edges into next 
    # exercise with priority and time
    m_ex = (exercise.maxWork // exercise.pause)
    i = 0
    while i < m_ex:
        for ex in exercise.allExercise.values():
            if ex.category == "warmup": # already did that
                continue
            graph[f"{ex.name} {i}"] = {}
            for ex2 in exercise.allExercise.values():
                if f"{ex2.name} {i+1}" not in graph_r:
                    graph_r[f"{ex2.name} {i+1}"] = {}
                if ex2.category == "warmup":
                    continue
                if i < m_ex - 1: 
                    graph[f"{ex.name} {i}"][f"{ex2.name} {i+1}"] = (ex2.priority, ex2.totalTime + exercise.pause)
                    graph_r[f"{ex2.name} {i+1}"][f"{ex.name} {i}"] = (ex2.priority, ex2.totalTime + exercise.pause)
                else: 
                    graph[f"{ex.name} {i}"][f"{ex2.name} {i+1}"] = (ex2.priority, ex2.totalTime)
                    graph_r[f"{ex2.name} {i+1}"][f"{ex.name} {i}"] = (ex2.priority, ex2.totalTime)
        i += 1

    # the last layer is just the last exercise
    for ex in exercise.allExercise.values():
        if ex.category == "warmup":
            continue
        graph[f"{ex.name} {i}"] = {}

    # finally, we connected the warumups to the first exercise layer
    for ex in exercise.allExercise.values():
        graph_r[f"{ex.name} 0"] = {}
        for warmup in warumups:
            if ex.category == "warmup":
                continue
            graph[f"{warmup.name}"][f"{ex.name} 0"] = (ex.priority, ex.totalTime + exercise.pause)
            graph_r[f"{ex.name} 0"][f"{warmup.name}"] = (ex.priority, ex.totalTime + exercise.pause)

    return graph, graph_r


def build_model(exercise_path, calender_path):
    ex = ExInfo(exercise_path)
    cal = UserCalendar(ex.zone) 
    cal.load_calendar(calender_path)
    day_graph, day_graph_r = build_day_graph(ex)
    for key, value in day_graph.items():
        # make sure that the graph matching to the reverse graph 
        assert key in day_graph_r, f"key {key} not in day_graph_r" 
        for key2, value2 in value.items():
            assert key2 in day_graph_r, f"key2 {key2} not in day_graph_r" 
            assert key in day_graph_r[key2], f"key {key} not in {key2}:{day_graph_r[key2]}"
    # example
    print(f"possible Exercises after Barbell Row 0: {day_graph['Barbell Row 0']}")
    print(f"possible Exercises before Barbell Row 0: {day_graph_r['Barbell Row 0']}")
    # todo: create interlinked graph for all days 
    days = list(zip(range(1, ex.weekNum * ex.weekLen + 1), range(2, ex.weekNum * ex.weekLen + 1)))
    day_combined_graph = {}
    day_combined_graph_r = {}
    for today, tomorrow in days:
        print(f"Day {today} -> {tomorrow}")
        day_combined_graph[today] = {}
        day_combined_graph_r[tomorrow] = {}

    

build_model("ex1.xlsx", "cal1.ics")


possible Exercises after Barbell Row 0: {'Abduction Machine 1': (3, 11), 'Adduction Machine 1': (3, 11), 'Barbell Row 1': (3, 11), 'Bench Press 1': (5, 15), 'Benchpress Declined Dunbells 1': (3, 11), 'Benchpress Inclined Dunbells 1': (4, 11), 'Biceps Curls Barbell 1': (2, 11), 'Biceps Curls Dunbells 1': (4, 11), 'Butterfly 1': (3, 11), 'Butterfly Reverse 1': (3, 11), 'Cable Crossover 1': (4, 11), 'Cable Row Narrow 1': (4, 11), 'Cable Row Wide 1': (3, 11), 'Calves Press Machine 1': (2, 11), 'Core Workout 1': (4, 11), 'Deadlift 1': (5, 12), 'Dips 1': (3, 11), 'Face Pulls 1': (3, 11), 'Hammer Curls 1': (2, 11), 'Hip Raises 1': (2, 11), 'Incline Biceps Curl 1': (3, 11), 'Lat Pull 1': (5, 13.5), 'Lateral Raises 1': (3, 11), 'Leg Curls Laying 1': (3, 11), 'Leg Curls Sitting 1': (3, 11), 'Leg Extension Machine 1': (3, 11), 'Lunges 1': (4, 11), 'Lying Triceps Extension 1': (2, 11), 'Overhead Press 1': (2, 11), 'Overhead Triceps Extensios 1': (2, 11), 'Prisoner Back Extensions 1': (2, 11), 'Pul

In [10]:
import gurobipy as gp
def solve(full_instance_path, calendar_path):
    pass